In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%%capture
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install sentencepiece

In [ ]:
from datasets import load_dataset
raw_datasets = load_dataset("indonlu", "smsa")

Generating train split:   0%|          | 0/11000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1260 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
raw_datasets["train"]

Dataset({
    features: ['text', 'label'],
    num_rows: 11000
})

In [ ]:
df = pd.DataFrame()
df["text"] = raw_datasets["validation"]["text"]
df['label'] = raw_datasets['validation']['label']
df.to_csv("valid.csv")

In [ ]:
from datasets import load_dataset
from transformers import BertTokenizer, DataCollatorWithPadding,AutoModel

raw_datasets = load_dataset("indonlu", "smsa")
checkpoint = "indobenchmark/indobert-lite-base-p1"
tokenizer = BertTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True,max_length = 256)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


Map:   0%|          | 0/11000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1260 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label","labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 11000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1260
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 500
    })
})

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-lite-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 49]),
 'token_type_ids': torch.Size([8, 49]),
 'attention_mask': torch.Size([8, 49])}

In [ ]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(0.9080, grad_fn=<NllLossBackward0>) torch.Size([8, 3])


In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(),lr = 2e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from transformers import get_scheduler

num_epochs = 5
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

6875


In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))


model.train()

for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs =model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
    print(f"\rloss : {outputs.loss}",end = " ")

  0%|          | 0/6875 [00:00<?, ?it/s]

loss : 0.0007213196367956698 

In [ ]:
import evaluate

metric = evaluate.load("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.9476190476190476}

In [ ]:
model.save_pretrained("indobert-lite-base-p1-smsa")
import shutil
shutil.make_archive("indobert-lite-base-p1-smsa", 'zip', "indobert-lite-base-p1-smsa")

'/content/indobert-lite-base-p1-smsa.zip'

In [ ]:
tokenizer.save_pretrained("tokenizer-indober-base-p1")
import shutil
shutil.make_archive("tokenizer-lite-base-p1","zip","tokenizer-indober-base-p1")

'/content/tokenizer-lite-base-p1.zip'

In [ ]:
pwd

'/content'

In [ ]:
model1 = AutoModelForSequenceClassification.from_pretrained('indobert-lite-base-p1-smsa')
tokenizer1 = AutoTokenizer.from_pretrained('tokenizer-indober-base-p1')

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the pre-trained model and tokenizer
# model_name = "your_model_name"  # Replace with the model name you're using
# model = AutoModelForSequenceClassification.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# Prepare an example input text
input_text = "mau tanya"

inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

# Perform inference
with torch.no_grad():
    outputs = model(**inputs)

# Extract the predicted class probabilities or labels
logits = outputs.logits
predicted_class = torch.argmax(logits, dim=-1).item()

# Print the predicted class label
print(f"Predicted class: {predicted_class}")


Predicted class: 1


In [ ]:
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

# Perform inference
with torch.no_grad():
    outputs = model(**inputs)

# Extract the predicted class probabilities or labels
logits = outputs.logits
predicted_class = torch.argmax(logits, dim=-1).item()

# Print the predicted class label
print(f"Predicted class: {predicted_class}")

Predicted class: 2


In [ ]:
def map_to_sentiment(predicted_class):
    if predicted_class == 2:
        return "negative"
    elif predicted_class == 1:
        return "neutral"
    elif predicted_class == 0:
        return "positive"
    else:
        return "unknown"

In [ ]:
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# # Load the pre-trained model and tokenizer
# model_name = "your_model_name"  # Replace with the model name you're using
# model = AutoModelForSequenceClassification.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create a DataFrame with text data
df = pd.read_csv('Preprocessed_MyRep_complete.csv',sep=";")

# df = pd.DataFrame(data2)

# Define a function to make predictions
def predict_class(text):
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=-1).item()
    sentiment = map_to_sentiment(predicted_class)

    return sentiment

# Apply the prediction function to each row in the DataFrame
df["tag"] = df["Filtered_Text"].apply(predict_class)

# Save the results to a new DataFrame or CSV file
result_df = df.copy()  # You can also save it to a new DataFrame
result_df.to_csv("result_tags_MyRep.csv", index=False)  # Save to a CSV file

# Print the resulting DataFrame
print(result_df)


                 UserName            Handle                 Timestamp  \
0                 ケッサワ ——         @ykessawa  2023-10-24T18:56:29.000Z   
1             RyoMiyaichi   @reynand_emilio  2023-10-24T09:44:10.000Z   
2    ranjau(ranger hijau)         @FakhriWs  2023-10-24T08:56:05.000Z   
3                      A.    @kumbanglincah  2023-10-23T13:22:13.000Z   
4                  dimdum       @dihmasasih  2023-10-22T05:43:33.000Z   
..                    ...               ...                       ...   
466           kopi nuklir  @kopikapalnuklir  2023-09-16T14:18:41.000Z   
467           wahyu I. f.       @fahroeddin  2023-09-14T14:29:35.000Z   
468       Azazel Basudara      @azazelious_  2023-09-12T11:50:00.000Z   
469           mba ataaak.   @b0mbaysapph1re  2023-09-12T09:47:30.000Z   
470           mba ataaak.   @b0mbaysapph1re  2023-09-12T08:03:47.000Z   

                                                  Text Emojis  Comments  \
0    kenapa pake \n@MyRepublicID\n nggk bisa buk

In [ ]:
df = pd.read_csv('result_tags_MyRep.csv',sep=",")


In [ ]:
# Assuming your DataFrame is named df and it has a "tag" column
value_counts = df["tag"].value_counts()

# Print or access the counts for each unique value
print(value_counts)

# If you want to access the count of a specific value, you can do so by indexing the result
# For example, to get the count for the "positive" tag:
positive_count = value_counts["positive"]
print(f"Count for 'positive': {positive_count}")
#My Rep

negative    342
neutral     105
positive     24
Name: tag, dtype: int64
Count for 'positive': 24


In [ ]:
# Assuming your DataFrame is named df and it has a "tag" column
value_counts = df["tag"].value_counts()

# Print or access the counts for each unique value
print(value_counts)

# If you want to access the count of a specific value, you can do so by indexing the result
# For example, to get the count for the "positive" tag:
positive_count = value_counts["positive"]
print(f"Count for 'positive': {positive_count}")
#IH

negative    350
neutral     113
positive     37
Name: tag, dtype: int64
Count for 'positive': 37


In [ ]:
pip install bertopic


In [ ]:
from bertopic import BERTopic


In [ ]:
def tokenisasi(df):
    stopwords = load_stopwords()
    tokenizer = AutoTokenizer.from_pretrained('indobert')
    tokens = df.content.apply(lambda x: tokenizer.tokenize(x))
    tokens = tokens.apply(lambda x: [x for x in x if (not x.startswith('##') and x not in stopwords and len(x) > 4)])
    return tokens

In [ ]:
kelas_sentiment=['positive','neutral', 'neutral' ]

In [ ]:
cmap_dict = {'positif': 'Greens', 'negatif': 'OrRd', 'netral': 'GnBu'}
tokens = tokenisasi(df[df.tag == kelas_sentiment])
tokens = tokens.apply(lambda x: ' '.join(x))
text = ' '.join(tokens)
wordcloud = WordCloud(width = 800, height = 800,
                background_color ='black',
                min_font_size = 10,
                colormap = cmap_dict[kelas_sentiment]).generate(text)

ValueError: ignored